# SEGUNDA AVALIAÇÃO DE ECONOMETRIA

## Importação das Bibliotecas

In [1]:
# Análise de dados básica

import pandas as pd
import numpy as np

# Visualização

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

# Econometria

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from linearmodels.iv import IV2SLS
from statsmodels.stats.stattools import durbin_watson 
from linearmodels.panel import PanelOLS
import linearmodels.panel as pl
from linearmodels import PanelOLS, RandomEffects
from linearmodels.panel import compare, PooledOLS
import statsmodels.api as sm

# Estatística

from scipy import stats

## Configuração do Display

In [2]:
# O pandas foi configurado para mostrar todas as colunas e as 100 primeiras linhas

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', 10) 

## Carregamento da Base de Dados

In [3]:
# O caminho do arquivo (base de dados) foi guardado em uma variável

caminho_arquivo = "/home/ph-padrim/Área de trabalho/Trabalho de Econometria/trabalho 2/painel_2011-2021_raisfirmasmg.csv"

# O arquivo CSV foi transformado em um dataframe

df = pd.read_csv(caminho_arquivo, encoding='UTF-8')  

# Criando a coluna com o quadrado da idade_med

df['idade_med_quadrado'] = df['idade_med'] ** 2

# Criando interação entre sexo e raça/cor. Se forem categóricas, primeiro crie dummies

df['sexo_raca_interacao'] = df['sexo_med'] * df['raca_cor_med']

# Criando dummies de nível técnico da firma

dummies = pd.get_dummies(df['nivel_tec'], prefix='nivel_tec')
dummies = dummies[[f'nivel_tec_{i}' for i in range(1, 6)]]  # Remove o nível 0
df = pd.concat([df, dummies], axis=1)

# Convertendo as dummies em inteiros

for nivel in range(1, 6):
    df[f'nivel_tec_{nivel}'] = (df['nivel_tec'] == nivel).astype(int)

# Cria variáveis dummy para cada ano no período 2011-2021. Exemplo: d_2011 = 1 se o ano é 2011, 0 caso contrário

years = range(2011, 2021+1)
for ano in years:
    df[f'd_{ano}'] = np.where(df['ano'] == ano, 1, 0)

# Dropando os anos que não possuem observação: 2012, 2014, 2016, 2018, 2020

df = df.drop(['d_2012', 'd_2014', 'd_2016', 'd_2018', 'd_2020'], axis=1)

df_index = df.set_index(['firma', 'ano'])
display(df_index)

# Contando quantos valores zerados da variável df['remun_med_real_med'] e excluindo 
print('Informações importantes:')
print('--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
quantidade_zeros = (df['remun_med_real_med'] == 0).sum()
print(f'1) Existem {quantidade_zeros} valores 0 da variável remun_med_real_med na base original da RAIS. Serão excluídos, pois são poucos (representam {(quantidade_zeros/393606)*100:.4}%) do total.')
df = df[df['remun_med_real_med'] != 0] 

# Contando quantos valores possuem 'NA' e excluindo 

print(f"2) Foram excluídas {390780 - 311862} linhas ({(78918/393606*100):.4}% do total) que possuíam algum NA nas variáveis utilizadas em algum momento da modelagem. Portanto, o painel será desbalanceado.")
print('--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')

# Logaritmizando o y e adicionando na base df

y_log = np.log(df['remun_med_real_med']) 
df['y_log'] = y_log

# Nome da coluna que você quer classificar
coluna_nivel_tecnologico = 'nivel_tec'

# Nome da nova coluna que será criada
nova_coluna_classificacao = 'nivel_tec_classificado'

# Verifica se a coluna 'nivel_tec' existe antes de tentar classificá-la
if coluna_nivel_tecnologico in df.columns:
    # Classifica a firma: 1 se 'nivel_tec' for 5, 0 caso contrário
    # Esta é a forma mais concisa e eficiente em Pandas
    df[nova_coluna_classificacao] = (df[coluna_nivel_tecnologico] == 5).astype(int)

    # Alternativa usando .apply (um pouco menos performática para grandes DFs, mas mais explícita na lógica)
    # df[nova_coluna_classificacao] = df[coluna_nivel_tecnologico].apply(lambda x: 1 if x == 5 else 0)

    print("\nDataFrame com a nova coluna de classificação (Diretamente no 'df'):")
    print(df)
    print("-" * 50)
else:
    print(f"Erro: A coluna '{coluna_nivel_tecnologico}' não foi encontrada no DataFrame.")

# Você pode salvar o DataFrame modificado se desejar
# df.to_csv('rais_modificada.csv', index=False)
# df.to_excel('rais_modificada.xlsx', index=False)

# Nome da coluna de ano
coluna_ano = 'ano'

# Nome da nova coluna que será criada
nova_coluna_classificacao_ano = 'ano_classificado'

# Verifica se a coluna 'ano' existe antes de tentar classificá-la
if coluna_ano in df.columns:
    # Classifica o ano: 1 se 'ano' for 2021, 0 caso contrário
    # Usa a comparação booleana e converte True/False para 1/0
    df[nova_coluna_classificacao_ano] = (df[coluna_ano] == 2021).astype(int)

    print("\nDataFrame com a nova coluna 'ano_classificado':")
    print(df)
    print("-" * 50)
else:
    print(f"Erro: A coluna '{coluna_ano}' não foi encontrada no DataFrame.")

# Criando interação entre dummy de nivel_tec e ano. Se forem categóricas, primeiro crie dummies

df['tec_ano_interacao'] = df['nivel_tec_classificado'] * df['ano_classificado']

# Define as variáveis explicativas

controle_vars = ["idade_med", "idade_med_quadrado", "sexo_med", "raca_cor_med", "superior_med", "sexo_raca_interacao", "nivel_tec_1", "nivel_tec_2", "nivel_tec_3", "nivel_tec_4", "nivel_tec_5"]

# Calcula médias por indivíduo (firma) para todas as variáveis. Isso é necessário para a abordagem de Mundlak

years2 = range(2011, 2021+1, 2)
for var in controle_vars + [f'd_{ano}' for ano in years2]:
    df[f'mean_{var}'] = df.groupby('firma')[var].transform('mean')

# Configura o índice do painel principal (df) usando a identificação da firma e o ano

df_index = df.set_index(['firma', 'ano'])

# Configura o índice do painel secundário (dados usados em algum momento na modelagem) usando a identificação da firma e o ano

dados_limpos = df[['firma', 'ano', 'y_log', 'adultos_med', 'idade_med', 'superior_med', 'idade_med_quadrado', 'sexo_med', 'raca_cor_med', 'sexo_raca_interacao', 'nivel_tec_1', 'nivel_tec_2', 'nivel_tec_3', 'nivel_tec_4', 'nivel_tec_5', 'ano_classificado', 'nivel_tec_classificado', 'tec_ano_interacao', 'remun_med_real_med']].dropna()
dados_limpos_index = dados_limpos.set_index(['firma', 'ano'])

sexo_med  raca_cor_med  tempo_emprego_med  idade_med  \
firma ano                                                          
1     2011  0.000000      1.000000           9.450000  25.500000   
      2021  1.000000      0.500000          53.300000  26.000000   
      2015  0.250000      0.750000          10.925000  22.250000   
      2019  0.400000      0.600000          40.939999  25.799999   
      2013  1.000000      0.000000           6.900000  23.000000   
...              ...           ...                ...        ...   
65601 2017  0.070746      0.183556          90.150864  36.430210   
      2019  0.078431      0.252941          94.140785  37.501961   
      2015  0.077535      0.210736          82.044731  35.035786   
      2021  0.126957           NaN          85.735652  37.086957   
      2013  0.072016      0.240741          74.588066  34.633743   

            remun_med_real_med    hb_med    hn_med    mb_med    mn_med  \
firma ano                                                                
1     2011          909.060543  0.000000  1.000000  0.000000  0.000000   
      2021         1651.425000  0.000000  0.000000  0.500000  0.500000   
      2015         1467.106445  0.000000  0.750000  0.250000  0.000000   
      2019         1251.502930  0.000000  0.600000  0.400000  0.000000   
      2013         1973.719360  0.000000  0.000000  1.000000  0.000000   
...                        ...       ...       ...       ...       ...   
65601 2017         4256.148438  0.751434  0.177820  0.065010  0.005736   
      2019         3932.223633  0.690196  0.231373  0.056863  0.021569   
      2015         4376.018555  0.719682  0.202783  0.069583  0.007952   
      2021         3385.715687       NaN       NaN       NaN       NaN   
      2013         4785.799316  0.699588  0.228395  0.059671  0.012346   

            envelhecidos_med  adultos_med  jovens_med  ate_fundamental_med  \
firma ano                                                                    
1     2011          0.000000     0.000000    1.000000             0.500000   
      2021          0.000000     0.000000    1.000000             0.000000   
      2015          0.000000     0.000000    1.000000             0.750000   
      2019          0.000000     0.200000    0.800000             0.200000   
      2013          0.000000     0.000000    1.000000             0.000000   
...                      ...          ...         ...                  ...   
65601 2017          0.036329     0.512428    0.451243             0.152964   
      2019          0.050980     0.554902    0.394118             0.131373   
      2015          0.037773     0.425447    0.536779             0.174950   
      2021          0.057391     0.540870    0.401739             0.120000   
      2013          0.034979     0.390947    0.574074             0.185185   

            superior_med  rotatividade_med  mismatchover_med  \
firma ano                                                      
1     2011      0.000000          0.000000          0.500000   
      2021      0.000000          0.000000          0.000000   
      2015      0.000000          0.000000          0.750000   
      2019      0.000000          0.000000          0.200000   
      2013      0.000000          0.000000          0.000000   
...                  ...               ...               ...   
65601 2017      0.135755          0.009560          0.172084   
      2019      0.209804          0.011765          0.147059   
      2015      0.161034          0.009940          0.182903   
      2021      0.215652          0.017391          0.135652   
      2013      0.176955          0.022634          0.156379   

            mismatchunder_med  mismatchnull_med  tempo_emprego_a1_med  \
firma ano                                                               
1     2011           0.000000          0.500000              0.500000   
      2021           0.000000          1.000000              0.500000   
      2015           0.0

Informações importantes:
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
1) Existem 2826 valores 0 da variável remun_med_real_med na base original da RAIS. Serão excluídos, pois são poucos (representam 0.718%) do total.
2) Foram excluídas 78918 linhas (20.05% do total) que possuíam algum NA nas variáveis utilizadas em algum momento da modelagem. Portanto, o painel será desbalanceado.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

DataFrame com a nova coluna de classificação (Diretamente no 'df'):
        sexo_med  raca_cor_med  tempo_emprego_med  idade_med  \
0       0.000000      1.000000           9.450000  25.500000   
1       1.000000      0.500000          53.300000  26.000000   
2       0.250000      

## Display Resumido do Dataframe

In [4]:
# Mostra as 10 primeiras linhas do dataframe

display(df)
display(dados_limpos_index[dados_limpos_index['tec_ano_interacao']==1])


,sexo_med,raca_cor_med,tempo_emprego_med,idade_med,remun_med_real_med,hb_med,hn_med,mb_med,mn_med,envelhecidos_med,adultos_med,jovens_med,ate_fundamental_med,superior_med,rotatividade_med,mismatchover_med,mismatchunder_med,mismatchnull_med,tempo_emprego_a1_med,tempo_emprego_2a5_med,tempo_emprego_6mais_med,sal_hora_med,cbo_h1_med,cbo_h2_med,cbo_h3_med,sal_hora_dp,produtividade,data_abertura,data_encerramento,municipio,natureza_juridica,qtd_vinc_atv,ibge_subset,ind_atv_ano,cnae_20,nivel_tec,tamanho,idade,tamanho_2,setor_2,regiao,ano,firma,idade_med_quadrado,sexo_raca_interacao,nivel_tec_1,nivel_tec_2,nivel_tec_3,nivel_tec_4,nivel_tec_5,d_2011,d_2013,d_2015,d_2017,d_2019,d_2021,y_log,nivel_tec_classificado,ano_classificado,tec_ano_interacao,mean_idade_med,mean_idade_med_quadrado,mean_sexo_med,mean_raca_cor_med,mean_superior_med,mean_sexo_raca_interacao,mean_nivel_tec_1,mean_nivel_tec_2,mean_nivel_tec_3,mean_nivel_tec_4,mean_nivel_tec_5,mean_d_2011,mean_d_2013,mean_d_2015,mean_d_2017,mean_d_2019,mean_d_2021
0,0.000000,1.000000,9.450000,25.500000,909.060543,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.500000,0.000000,0.000000,0.500000,0.000000,0.500000,0.500000,0.500000,0.000000,5.165117,0.000000,0,1.000000,0.379776,20.660467,20052008,0,313930,2135,1,16,1,45,1,1,4.0,1.0,3.0,3.0,2011,1,650.250000,0.000000,1,0,0,0,0,1,0,0,0,0,0,6.812412,0,0,0,24.425000,598.658743,0.497222,0.586111,0.000000,0.191620,0.333333,0.500000,0.0,0.000000,0.0,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667
1,1.000000,0.500000,53.300000,26.000000,1651.425000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.500000,0.000000,0.500000,9.383097,0.500000,NaN,0.500000,4.946493,37.532386,20052008,0,313930,2135,2,16,1,45,2,2,NaN,1.0,3.0,3.0,2021,1,676.000000,0.500000,0,1,0,0,0,0,0,0,0,0,1,7.409394,0,1,0,24.425000,598.658743,0.497222,0.586111,0.000000,0.191620,0.333333,0.500000,0.0,0.000000,0.0,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667
2,0.250000,0.750000,10.925000,22.250000,1467.106445,0.000000,0.750000,0.250000,0.000000,0.000000,0.000000,1.000000,0.750000,0.000000,0.000000,0.750000,0.000000,0.250000,0.750000,0.250000,0.000000,8.335832,0.000000,NaN,0.750000,2.793418,33.343327,20052008,0,313930,2135,4,16,1,45,1,2,NaN,NaN,NaN,NaN,2015,1,495.062500,0.187500,1,0,0,0,0,0,0,1,0,0,0,7.291047,0,0,0,24.425000,598.658743,0.497222,0.586111,0.000000,0.191620,0.333333,0.500000,0.0,0.000000,0.0,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667
3,0.400000,0.600000,40.939999,25.799999,1251.502930,0.000000,0.600000,0.400000,0.000000,0.000000,0.200000,0.800000,0.200000,0.000000,0.000000,0.200000,0.000000,0.800000,0.200000,0.600000,0.200000,7.744656,0.000000,NaN,0.800000,2.955299,31.603609,20052008,0,313930,2135,4,16,1,45,2,2,NaN,NaN,NaN,NaN,2019,1,665.639961,0.240000,0,1,0,0,0,0,0,0,0,1,0,7.132100,0,0,0,24.425000,598.658743,0.497222,0.586111,0.000000,0.191620,0.333333,0.500000,0.0,0.000000,0.0,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667
4,1.000000,0.000000,6.900000,23.000000,1973.719360,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,11.214314,0.000000,NaN,0.000000,NaN,44.857258,20052008,0,313930,2135,1,16,1,45,0,1,NaN,NaN,NaN,NaN,2013,1,529.000000,0.000000,0,0,0,0,0,0,1,0,0,0,0,7.587675,0,0,0,24.425000,598.658743,0.497222,0.586111,0.000000,0.191620,0.333333,0.500000,0.0,0.000000,0.0,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393601,0.070746,0.183556,90.150864,36.430210,4256.148438,0.751434,0.177820,0.065010,0.005736,0.036329,0.512428,0.451243,0.152964,0.13

,,y_log,adultos_med,idade_med,superior_med,idade_med_quadrado,sexo_med,raca_cor_med,sexo_raca_interacao,nivel_tec_1,nivel_tec_2,nivel_tec_3,nivel_tec_4,nivel_tec_5,ano_classificado,nivel_tec_classificado,tec_ano_interacao,remun_med_real_med
firma,ano,,,,,,,,,,,,,,,,,
161,2021,7.088937,0.333333,46.666667,0.000000,2177.777778,0.500000,1.000000,0.500000,0,0,0,0,1,1,1,1,1198.633333
273,2021,8.040411,0.714286,39.000000,0.142857,1521.000000,0.714286,0.428571,0.306122,0,0,0,0,1,1,1,1,3103.890000
670,2021,7.691000,0.000000,25.000000,0.000000,625.000000,0.066667,0.533333,0.035556,0,0,0,0,1,1,1,1,2188.561333
1207,2021,7.060442,0.000000,27.666667,0.000000,765.444444,0.333333,0.000000,0.000000,0,0,0,0,1,1,1,1,1164.960000
1208,2021,7.105195,0.363636,34.227273,0.909091,1171.506198,0.500000,0.000000,0.000000,0,0,0,0,1,1,1,1,1218.280000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65362,2021,8.373584,0.416667,38.250000,0.833333,1463.062500,0.500000,0.000000,0.000000,0,0,0,0,1,1,1,1,4331.129167
65363,2021,8.524236,0.545455,38.772727,0.818182,1503.324380,0.500000,0.136364,0.068182,0,0,0,0,1,1,1,1,5035.338182
65393,2021,8.469127,0.272727,32.636364,0.818182,1065.132231,0.636364,0.363636,0.231405,0,0,0,0,1,1,1,1,4765.351818


## Análise Exploratória

### Descrição das Variáveis

In [5]:
# Gera estatísticas descritivas das principais variáveis

print('--------------------------------------------------------------------------')
desc_stats = dados_limpos.describe()
print(desc_stats)
print('--------------------------------------------------------------------------')

# Supondo que df seja seu DataFrame
quantidade_zeros = (df['remun_med_real_med'] == 0).sum()

print(f"Existem {quantidade_zeros} valores zerados na coluna remun_med_real_med")


--------------------------------------------------------------------------
               firma            ano          y_log    adultos_med  \
count  311862.000000  311862.000000  311862.000000  311862.000000   
mean    32942.299100    2015.611751       7.361490       0.387261   
std     18939.566705       3.365059       0.431398       0.289893   
min         1.000000    2011.000000       2.486937       0.000000   
25%     16528.000000    2013.000000       7.080621       0.166667   
50%     33043.500000    2015.000000       7.262308       0.363636   
75%     49546.000000    2019.000000       7.540576       0.534247   
max     65601.000000    2021.000000      11.754593       1.000000   

           idade_med   superior_med  idade_med_quadrado       sexo_med  \
count  311862.000000  311862.000000       311862.000000  311862.000000   
mean       35.218499       0.087056         1300.904121       0.463178   
std         7.782137       0.195953          595.960079       0.364316   
min    

### Analisando modelos

#### Modelo Pooled

In [6]:
# Criação das variáveis X e Y utilizadas nos modelos

X_sem_constante = dados_limpos_index.drop(columns=['y_log', 'adultos_med'])
X = sm.add_constant(X_sem_constante[['idade_med', 'superior_med', 'idade_med_quadrado', 'sexo_med', 'raca_cor_med', 'sexo_raca_interacao', 'nivel_tec_1', 'nivel_tec_2', 'nivel_tec_3', 'nivel_tec_4', 'nivel_tec_5']])
Y = dados_limpos_index[['y_log']]

# ==============================================
# 1. MODELO POOLED OLS
# ==============================================
modelo_pooled = PooledOLS(dependent=Y, exog=X).fit()
print(modelo_pooled.summary)

# ==============================================
# 2. TESTE DE HETEROCEDASTICIDADE (BREUSCH-PAGAN)
# ==============================================
residuals = modelo_pooled.resids.values.flatten()

if isinstance(X, pd.DataFrame):
    exog = X.values
else:
    try:
        exog = X.dataframe.values
    except AttributeError:
        exog = X.values

teste_bp = het_breuschpagan(residuals, exog)

labels_bp = ['Estatística LM', 'Valor-p LM', 'Estatística F', 'Valor-p F']
resultados_bp = dict(zip(labels_bp, teste_bp))

print("\n=== TESTE DE BREUSCH-PAGAN ===")
for key, value in resultados_bp.items():
    print(f"{key}: {value:.4f}")

# ==============================================
# 3. TESTE DE AUTOCORRELAÇÃO
# ==============================================
# Para PooledOLS, verificamos autocorrelação geral (não por entidade)
residuals = modelo_pooled.resids
resid_df = pd.DataFrame({'residuos': residuals})

# Calcular estatística de Durbin-Watson
dw = np.sum(np.diff(residuals)**2) / np.sum(residuals**2)

print("\n=== TESTE DE AUTOCORRELAÇÃO ===")
print(f"Estatística de Durbin-Watson: {dw:.4f}")

# Interpretação
print("\nInterpretação:")
print("≈2: sem autocorrelação")
print("<1.5: possível autocorrelação positiva")
print(">2.5: possível autocorrelação negativa")

# ==============================================
# 4. TESTE DE MULTICOLINEARIDADE (VIF)
# ==============================================
if isinstance(X, pd.DataFrame):
    vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    print("\n=== FATOR DE INFLACÃO DE VARIÂNCIA (VIF) ===")
    for i, col in enumerate(X.columns):
        print(f"{col}: VIF = {vif[i]:.2f}")
else:
    print("\nAVISO: VIF não calculado - X precisa ser um DataFrame")

# ==============================================
# 5. VERSÃO COM ERROS ROBUSTOS (OPCIONAL)
# ==============================================
print("\n=== VERSÃO COM ERROS ROBUSTOS ===")
modelo_robusto = PooledOLS(dependent=Y, exog=X).fit(cov_type='robust')
print(modelo_robusto.summary)


                          PooledOLS Estimation Summary                          
Dep. Variable:                  y_log   R-squared:                        0.3256
Estimator:                  PooledOLS   R-squared (Between):              0.3940
No. Observations:              311862   R-squared (Within):              -0.1618
Date:                Fri, Jun 27 2025   R-squared (Overall):              0.3256
Time:                        08:02:09   Log-likelihood                -1.189e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                   1.369e+04
Entities:                       63239   P-value                           0.0000
Avg Obs:                       4.9315   Distribution:               F(11,311850)
Min Obs:                       1.0000                                           
Max Obs:                       6.0000   F-statistic (robust):          1.369e+04
                            

#### Modelo considerando os efeitos fixos

In [7]:
# ==============================================
# 1. MODELO DE EFEITOS FIXOS
# ==============================================
efeitos_fixos = PanelOLS(Y, X, entity_effects=True).fit()
print(efeitos_fixos.summary)

# ==============================================
# 2. TESTE DE HETEROCEDASTICIDADE (BREUSCH-PAGAN)
# ==============================================
residuals = efeitos_fixos.resids.values.flatten()

if isinstance(X, pd.DataFrame):
    exog = X.values
else:
    try:
        exog = X.dataframe.values
    except AttributeError:
        exog = X.values

teste_bp = het_breuschpagan(residuals, exog)

labels_bp = ['Estatística LM', 'Valor-p LM', 'Estatística F', 'Valor-p F']
resultados_bp = dict(zip(labels_bp, teste_bp))

print("\n=== TESTE DE BREUSCH-PAGAN ===")
for key, value in resultados_bp.items():
    print(f"{key}: {value:.4f}")

# ==============================================
# 3. TESTE DE AUTOCORRELAÇÃO SIMPLIFICADO
# ==============================================
# Extrair resíduos e índices diretamente
residuals = efeitos_fixos.resids
resid_df = residuals.to_frame(name='residuos')
resid_df['entidade'] = resid_df.index.get_level_values(0)  # Assume que o primeiro nível é a entidade

# Calcular autocorrelação por entidade
autocorr_results = resid_df.groupby('entidade')['residuos'].apply(
    lambda x: x.autocorr(lag=1)  # Autocorrelação de 1ª ordem
)

print("\n=== TESTE DE AUTOCORRELAÇÃO ===")
print("Autocorrelação de 1ª ordem por entidade:")
print(autocorr_results)
print(f"\nAutocorrelação média: {autocorr_results.mean():.4f}")

# Interpretação
print("\nInterpretação:")
print("Valor próximo de 0: sem autocorrelação")
print("Valor positivo (>0.2): autocorrelação positiva")
print("Valor negativo (<-0.2): autocorrelação negativa")

# ==============================================
# 4. TESTE DE MULTICOLINEARIDADE (VIF)
# ==============================================
if isinstance(X, pd.DataFrame):
    vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    print("\n=== FATOR DE INFLACÃO DE VARIÂNCIA (VIF) ===")
    for i, col in enumerate(X.columns):
        print(f"{col}: VIF = {vif[i]:.2f}")
else:
    print("\nAVISO: VIF não calculado - X precisa ser um DataFrame")

# ==============================================
# 5. MODELO DE EFEITOS FIXOS (CORRIGIDO)
# ==============================================
efeitos_fixos = PanelOLS(Y, X, entity_effects=True).fit(cov_type='clustered', cluster_entity=True)
print(efeitos_fixos.summary)


                          PanelOLS Estimation Summary                           
Dep. Variable:                  y_log   R-squared:                        0.0589
Estimator:                   PanelOLS   R-squared (Between):              0.1765
No. Observations:              311862   R-squared (Within):               0.0589
Date:                Fri, Jun 27 2025   R-squared (Overall):              0.1635
Time:                        08:02:18   Log-likelihood                 1.278e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      1413.6
Entities:                       63239   P-value                           0.0000
Avg Obs:                       4.9315   Distribution:               F(11,248612)
Min Obs:                       1.0000                                           
Max Obs:                       6.0000   F-statistic (robust):             1413.6
                            

/home/ph-padrim/Área de trabalho/Trabalho de Econometria/trabalho 2/novo_venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3055: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/home/ph-padrim/Área de trabalho/Trabalho de Econometria/trabalho 2/novo_venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2912: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/home/ph-padrim/Área de trabalho/Trabalho de Econometria/trabalho 2/novo_venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2912: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)



=== TESTE DE AUTOCORRELAÇÃO ===
Autocorrelação de 1ª ordem por entidade:
entidade
1       -0.644078
2       -0.257701
4       -1.000000
5       -0.880772
6       -0.530960
           ...   
65597   -0.252398
65598    0.474782
65599   -0.028057
65600   -0.475670
65601    0.065501
Name: residuos, Length: 63239, dtype: float64

Autocorrelação média: -0.2083

Interpretação:
Valor próximo de 0: sem autocorrelação
Valor positivo (>0.2): autocorrelação positiva
Valor negativo (<-0.2): autocorrelação negativa

=== FATOR DE INFLACÃO DE VARIÂNCIA (VIF) ===
const: VIF = 259.08
idade_med: VIF = 44.29
superior_med: VIF = 1.05
idade_med_quadrado: VIF = 44.04
sexo_med: VIF = 1.87
raca_cor_med: VIF = 2.44
sexo_raca_interacao: VIF = 3.16
nivel_tec_1: VIF = 1.19
nivel_tec_2: VIF = 1.16
nivel_tec_3: VIF = 1.04
nivel_tec_4: VIF = 1.05
nivel_tec_5: VIF = 1.02
                          PanelOLS Estimation Summary                           
Dep. Variable:                  y_log   R-squared:                 

#### Modelo considerando os efeitos aleatórios

In [8]:
# ==============================================
# 1. MODELO DE EFEITOS ALEATÓRIOS
# ==============================================
modelo_re = RandomEffects(Y, X).fit()
print(modelo_re.summary)

# ==============================================
# 2. TESTE DE HETEROCEDASTICIDADE (BREUSCH-PAGAN)
# ==============================================
residuals = modelo_re.resids.values.flatten()

if isinstance(X, pd.DataFrame):
    exog = X.values
else:
    try:
        exog = X.dataframe.values
    except AttributeError:
        exog = X.values

teste_bp = het_breuschpagan(residuals, exog)

labels_bp = ['Estatística LM', 'Valor-p LM', 'Estatística F', 'Valor-p F']
resultados_bp = dict(zip(labels_bp, teste_bp))

print("\n=== TESTE DE BREUSCH-PAGAN ===")
for key, value in resultados_bp.items():
    print(f"{key}: {value:.4f}")

# ==============================================
# 3. TESTE DE AUTOCORRELAÇÃO
# ==============================================
# Extrair resíduos e índices
residuals = modelo_re.resids
resid_df = residuals.to_frame(name='residuos')
resid_df['entidade'] = resid_df.index.get_level_values(0)  # Assume primeiro nível é entidade

# Calcular autocorrelação por entidade
autocorr_results = resid_df.groupby('entidade')['residuos'].apply(
    lambda x: x.autocorr(lag=1)  # Autocorrelação de 1ª ordem
)

print("\n=== TESTE DE AUTOCORRELAÇÃO ===")
print("Autocorrelação de 1ª ordem por entidade:")
print(autocorr_results)
print(f"\nAutocorrelação média: {autocorr_results.mean():.4f}")

# Interpretação
print("\nInterpretação:")
print("Valor próximo de 0: sem autocorrelação")
print("Valor positivo (>0.2): autocorrelação positiva")
print("Valor negativo (<-0.2): autocorrelação negativa")

# ==============================================
# 4. TESTE DE MULTICOLINEARIDADE (VIF)
# ==============================================
if isinstance(X, pd.DataFrame):
    vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    print("\n=== FATOR DE INFLACÃO DE VARIÂNCIA (VIF) ===")
    for i, col in enumerate(X.columns):
        print(f"{col}: VIF = {vif[i]:.2f}")
else:
    print("\nAVISO: VIF não calculado - X precisa ser um DataFrame")

# ==============================================
# 5. MODELO DE EFEITOS ALEATÓRIOS (CORRIGIDO)
# ==============================================
modelo_re = RandomEffects(Y, X).fit(cov_type='clustered', cluster_entity=True)
print(modelo_re.summary)

                        RandomEffects Estimation Summary                        
Dep. Variable:                  y_log   R-squared:                        0.7305
Estimator:              RandomEffects   R-squared (Between):              0.2657
No. Observations:              311862   R-squared (Within):               0.0434
Date:                Fri, Jun 27 2025   R-squared (Overall):              0.2403
Time:                        08:02:41   Log-likelihood                 8.341e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                   7.685e+04
Entities:                       63239   P-value                           0.0000
Avg Obs:                       4.9315   Distribution:               F(11,311850)
Min Obs:                       1.0000                                           
Max Obs:                       6.0000   F-statistic (robust):             3416.4
                            

/home/ph-padrim/Área de trabalho/Trabalho de Econometria/trabalho 2/novo_venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3055: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/home/ph-padrim/Área de trabalho/Trabalho de Econometria/trabalho 2/novo_venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2912: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/home/ph-padrim/Área de trabalho/Trabalho de Econometria/trabalho 2/novo_venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:2912: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)



=== TESTE DE AUTOCORRELAÇÃO ===
Autocorrelação de 1ª ordem por entidade:
entidade
1       -0.639960
2       -0.261381
4       -1.000000
5       -0.927560
6       -0.567297
           ...   
65597   -0.393908
65598    0.117879
65599   -0.076308
65600   -0.446626
65601    0.011395
Name: residuos, Length: 63239, dtype: float64

Autocorrelação média: -0.2094

Interpretação:
Valor próximo de 0: sem autocorrelação
Valor positivo (>0.2): autocorrelação positiva
Valor negativo (<-0.2): autocorrelação negativa

=== FATOR DE INFLACÃO DE VARIÂNCIA (VIF) ===
const: VIF = 259.08
idade_med: VIF = 44.29
superior_med: VIF = 1.05
idade_med_quadrado: VIF = 44.04
sexo_med: VIF = 1.87
raca_cor_med: VIF = 2.44
sexo_raca_interacao: VIF = 3.16
nivel_tec_1: VIF = 1.19
nivel_tec_2: VIF = 1.16
nivel_tec_3: VIF = 1.04
nivel_tec_4: VIF = 1.05
nivel_tec_5: VIF = 1.02
                        RandomEffects Estimation Summary                        
Dep. Variable:                  y_log   R-squared:                 

#### Teste entre Pooled x Efeitos fixos

In [9]:
from linearmodels.panel import compare  
pvalue_ftest = efeitos_fixos.f_pooled  # p-valor do teste F  
print(f"P-valor (FE vs. Pooled): {pvalue_ftest}")  

# Será usado efeitos fixos

P-valor (FE vs. Pooled): Pooled F-statistic
H0: Effects are zero
Statistic: 15.1947
P-value: 0.0000
Distributed: F(63238,248612)


#### Teste de Hausman (Efeitos fixos X Efeitos aleatórios)

In [10]:
hausman = compare({'FE': efeitos_fixos, 'RE': modelo_re}, precision='tstats')
print(hausman)

# Será usado efeitos fixos

                    Model Comparison                   
                                   FE                RE
-------------------------------------------------------
Dep. Variable                   y_log             y_log
Estimator                    PanelOLS     RandomEffects
No. Observations               311862            311862
Cov. Est.                   Clustered         Clustered
R-squared                      0.0589            0.7305
R-Squared (Within)             0.0589            0.0434
R-Squared (Between)            0.1765            0.2657
R-Squared (Overall)            0.1635            0.2403
F-statistic                    1413.6         7.685e+04
P-value (F-stat)               0.0000            0.0000
=====================     ===========   ===============
const                          6.7884            6.7197
                             (533.90)          (560.27)
idade_med                      0.0275            0.0310
                             (38.658)          (

### Instrumentalização da variável superior_med e minimos quadrados em dois estágios

In [11]:
from linearmodels.iv import IV2SLS
import statsmodels.formula.api as smf

# --- 1. Modelo IV (2SLS) ---
model_iv = IV2SLS.from_formula(
    'y_log ~ 1 + idade_med + idade_med_quadrado + sexo_med + raca_cor_med + '
    'nivel_tec_1 + nivel_tec_2 + nivel_tec_3 + nivel_tec_4 + nivel_tec_5 '
    '+ [superior_med ~ adultos_med]', 
    data=df
)
results_iv = model_iv.fit(cov_type='robust')
print(results_iv.summary)

# --- 2. Verificação do Instrumento ---
print("\n=== TESTE DE RELEVÂNCIA DO INSTRUMENTO ===")

# Rodar regressão

base = df[['y_log', 'adultos_med', 'idade_med', 'superior_med', 'idade_med_quadrado', 'sexo_med', 'raca_cor_med', 'sexo_raca_interacao', 'nivel_tec_1', 'nivel_tec_2', 'nivel_tec_3', 'nivel_tec_4', 'nivel_tec_5']].dropna()
X = base[['adultos_med', 'idade_med', 'superior_med', 'idade_med_quadrado', 'sexo_med', 'raca_cor_med', 'sexo_raca_interacao', 'nivel_tec_1', 'nivel_tec_2', 'nivel_tec_3', 'nivel_tec_4', 'nivel_tec_5']]
Y = base[['y_log']]

first_stage_manual = sm.OLS(Y, X).fit()

# Obter resultados
f_stat = first_stage_manual.fvalue
coef_instrumento = first_stage_manual.params['adultos_med']

print(f"\n=== RESULTADOS MANUAIS DO 1º ESTÁGIO ===")
print(f"Estatística F: {f_stat:.4f}")
print(f"Coeficiente do instrumento: {coef_instrumento:.4f}")
print(f"Valor-p do instrumento: {first_stage_manual.pvalues['adultos_med']:.4f}")

if f_stat > 10:
    print("✅ Instrumento forte (F > 10)")
else:
    print("⚠️ Atenção: Instrumento potencialmente fraco (F ≤ 10)")

# --- 3. Modelo MQO para Hausman (DEVE USAR smf.ols) ---
model_ols = smf.ols(
    'y_log ~ 1 + superior_med + idade_med + idade_med_quadrado + '
    'sexo_med + raca_cor_med + nivel_tec_1 + nivel_tec_2 + nivel_tec_3 + nivel_tec_4 + nivel_tec_5', 
    data=df
)
results_ols = model_ols.fit(cov_type='HC3')  # Erros robustos
print("\n--------------------------------------------")
print(results_ols.summary())

# --- 4. Teste de Hausman ---
hausman_test = results_iv.wu_hausman()
print("\n=== TESTE DE HAUSMAN ===")
print(f"Estatística: {hausman_test.stat:.4f}")
print(f"p-valor: {hausman_test.pval:.4f}")

if hausman_test.pval < 0.05:
    print("✅ Rejeita H0: 'superior_med' é endógena (use IV2SLS).")
else:
    print("✅ Não rejeita H0: 'superior_med' pode ser exógena (MQO é válido).")

/home/ph-padrim/Área de trabalho/Trabalho de Econometria/trabalho 2/novo_venv/lib/python3.12/site-packages/linearmodels/iv/model.py:557: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


                          IV-2SLS Estimation Summary                          
Dep. Variable:                  y_log   R-squared:                     -0.0542
Estimator:                    IV-2SLS   Adj. R-squared:                -0.0542
No. Observations:              311862   F-statistic:                 3.335e+04
Date:                Fri, Jun 27 2025   P-value (F-stat)                0.0000
Time:                        08:03:13   Distribution:                 chi2(10)
Cov. Estimator:                robust                                         
                                                                              
                                 Parameter Estimates                                  
                    Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------------
Intercept              6.7588     0.0294     229.95     0.0000      6.7012      6.8164
idade_med           

#### DIFF IN DIFF

In [12]:
# --- Dividir o DataFrame 'df' em dois novos DataFrames ---

# DataFrame onde 'ano_classificado' é igual a 1
df_ano_1 = dados_limpos_index[dados_limpos_index['ano_classificado'] == 1].copy()

# DataFrame onde 'ano_classificado' é igual a 0
df_ano_0 = dados_limpos_index[dados_limpos_index['ano_classificado'] == 0].copy()

# ==============================================
# 1. MODELO POOLED OLS PARA O ANO 2021
# ==============================================
dados_limpos_index_exogenas = df_ano_1[['nivel_tec_classificado']]
exogenas_com_intercepto = sm.add_constant(dados_limpos_index_exogenas)
modelo_pooled = PooledOLS(dependent=df_ano_1['y_log'], exog=exogenas_com_intercepto).fit()
print(modelo_pooled.summary)

# ================================================
# 1. MODELO POOLED OLS PARA ANOS ANTERIORES A 2021
# ================================================
dados_limpos_index_exogenas = df_ano_0[['nivel_tec_classificado']]
exogenas_com_intercepto = sm.add_constant(dados_limpos_index_exogenas)
modelo_pooled = PooledOLS(dependent=df_ano_0['y_log'], exog=exogenas_com_intercepto).fit()
print(modelo_pooled.summary)

# ================================================
# 1. MODELO POOLED OLS CONSIDERANDO A INTERAÇÃO
# ================================================ 
dados_limpos_index_exogenas = dados_limpos_index[['tec_ano_interacao', 'ano_classificado', 'nivel_tec_classificado']]
exogenas_com_intercepto = sm.add_constant(dados_limpos_index_exogenas)

modelo_pooled = PooledOLS(dependent=dados_limpos_index['y_log'], exog=exogenas_com_intercepto).fit()
print(modelo_pooled.summary)





                          PooledOLS Estimation Summary                          
Dep. Variable:                  y_log   R-squared:                     8.041e-05
Estimator:                  PooledOLS   R-squared (Between):           8.041e-05
No. Observations:               42031   R-squared (Within):               0.0000
Date:                Fri, Jun 27 2025   R-squared (Overall):           8.041e-05
Time:                        08:03:21   Log-likelihood                -2.604e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3.3799
Entities:                       42031   P-value                           0.0660
Avg Obs:                       1.0000   Distribution:                 F(1,42029)
Min Obs:                       1.0000                                           
Max Obs:                       1.0000   F-statistic (robust):             3.3799
                            